# Demonstrate Interaction of pandas Dataframes and omnikeeper

In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import uuid
import pandas as pd

In [2]:
trait_id = "python_client_demo.test"
layer_id = "testlayer"

create a omnikeeper client

In [3]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [4]:
ret = okc.upsert_trait(okapiclient, TraitDefinition(trait_id, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))
assert(ret)

ensure layer exists

In [5]:
ret = okc.create_layer(okapiclient, layer_id, "just a test layer", okc.hex_string_to_rgb_color("#6666FF"))
assert(ret)

create a test DataFrame

In [6]:
df_init = pd.DataFrame.from_records([
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
])

df_init

,id,array,some_key
0,1,"[a, b]",Value 1
1,3,"[c, d]",another Value


write data to layer ``testlayer``

In [7]:
ret = okc.bulk_replace_trait_entities_by_filter_dataframe(okapiclient, trait_name=trait_id, input=df_init, id_attributes=["id"], id_relations=[], write_layer=layer_id, filter={})
assert(ret)

load data again from omnikeeper

- get that data back out from omnikeeper
- what we get back has an index that's the CIID of the corresponding CI
- we can choose if we want to keep the ciid as column, in this case we will not keep it

In [8]:
updated_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=trait_id, layers=[layer_id])
updated_df

,id,array,some_key
ciid,,,
2b321321-f307-47ba-970f-95486ed15bb1,3,"[c, d]",another Value
4d462469-a3c4-4f8c-be1f-020f64ddc142,1,"[a, b]",Value 1


modify our dataframe

In [9]:
# change things in the returned data
# change single value of existing data
updated_df.iat[0, updated_df.columns.get_loc("array")] = ["a", "b", "z"]
# add a new row, using our own created CIID
updated_df = pd.concat([updated_df, pd.DataFrame([{"id": 4, "array": ["x"], "some_key": "the new guy"}], index=[str(uuid.uuid4())])])
# drop a row
updated_df.drop([updated_df.index[1]], inplace = True)

updated_df

,id,array,some_key
2b321321-f307-47ba-970f-95486ed15bb1,3,"[a, b, z]",another Value
9b27175d-2af7-48da-a272-8964efeb006d,4,[x],the new guy


write it back to omnikeeper

In [10]:
ret = okc.bulk_replace_trait_entities_dataframe(okapiclient, trait_name=trait_id, input=updated_df, write_layer=layer_id)
assert(ret)

Fetch again the data

In [11]:
final_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=trait_id, layers=[layer_id])
final_df

,id,array,some_key
ciid,,,
2b321321-f307-47ba-970f-95486ed15bb1,3,"[a, b, z]",another Value
9b27175d-2af7-48da-a272-8964efeb006d,4,[x],the new guy


In [12]:
# cleanup
okc.delete_trait(okapiclient, trait_id)
okc.truncate_layer(okapiclient, layer_id)

True